<a href="https://colab.research.google.com/github/ksong1009/capstone/blob/main/Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium pandas  # Install folium and pandas using pip
import folium  # Import folium
import pandas as pd  # Import pandas
import requests # Import the requests library to make HTTP requests
import io
from folium.plugins import MarkerCluster # Import MarkerCluster from folium.plugins
from folium.plugins import MousePosition # Import MousePosition from folium.plugins
!pip install geopy
from geopy.distance import geodesic # Import geodesic from geopy.distance


# Download and read the `spacex_launch_geo.csv`
# Instead of using fetch from 'js', use the requests library to download the file.
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Use requests to get the content and read it with pandas
# The previous attempt with 'js' and 'fetch' is incorrect in Python.
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

!pip install folium pandas  # Install folium and pandas using pip
import folium  # Import folium
import pandas as pd  # Import pandas
import requests # Import the requests library to make HTTP requests
import io

# Download and read the `spacex_launch_geo.csv`
# Instead of using fetch from 'js', use the requests library to download the file.
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
response.raise_for_status()  # Raise an exception for bad responses (4xx or 5xx)

# Use requests to get the content and read it with pandas
# The previous attempt with 'js' and 'fetch' is incorrect in Python.
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

# Task 2: Mark the success/failed launches for each site on the map
#Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not
#circle=folium.Circle(class) #This line was causing the error.  It's removed as it's not valid code.  You will need to replace it with your actual code to plot the launches.

# Add markers for each launch
for _, row in spacex_df.iterrows():
    launch_site = row['Launch Site']
    lat, lon = row['Lat'], row['Long']
    success = row['class']

    # Set color based on success (green) or failure (red)
    color = "green" if success == 1 else "red"
    icon_text = "✅" if success == 1 else "❌"

    # Add marker
    folium.Marker(
        [lat, lon],
        popup=f"{launch_site} - {'Success' if success == 1 else 'Failure'}",
        icon=folium.Icon(color=color, icon="info-sign")
    ).add_to(site_map)

# Display the map
site_map

spacex_df.tail(10)

marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

# TASK 3: Calculate the distances between a launch site to its proximities
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

    # Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

# Assuming 'coordinates' will contain a list of coordinate pairs
# such as [[lat1, lon1], [lat2, lon2], ...]
# Replace this example with your actual coordinates

coordinates = [[29.559684888503615, -95.0830971930759], [34.632834, -120.610749]]

# Uncomment the line below and provide the 'coordinates' variable
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Define closest locations (for example)
closest_locations = {
    "City": {"name": "Houston", "coords": (29.7604, -95.3698)},
    "Railway": {"name": "Railway Station X", "coords": (29.7800, -95.3600)},
    "Highway": {"name": "Highway I-45", "coords": (29.7500, -95.3700)}
}

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Add launch site markers
for _, row in spacex_df.iterrows():
    launch_site = row['Launch Site']
    lat, lon = row['Lat'], row['Long']

    folium.Marker(
        [lat, lon],
        popup=f"{launch_site} - Launch Site",
        icon=folium.Icon(color="blue", icon="rocket")
    ).add_to(site_map)

    # Add closest locations and draw lines
for key, value in closest_locations.items():
        closest_name = value["name"]
        closest_coords = value["coords"]

        # Calculate distance
distance_km = geodesic((lat, lon), closest_coords).km

        # Add marker for closest location
folium.Marker(
            closest_coords,
            popup=f"{closest_name} - {key} ({distance_km:.2f} km)",
            icon=folium.Icon(color="green", icon="info-sign")
        ).add_to(site_map)

        # Draw line from launch site to closest location
folium.PolyLine([(lat, lon), closest_coords], color="red", weight=2.5, opacity=1).add_to(site_map)

# Display the map
site_map


